In [3]:
import getpass
from pprint import pprint  # more readable prints for complex structures

# this is similar to input(), but does not echo the typed text back
password = getpass.getpass()
MYSQL_PARAMS = {
    "database": "G7_Phone_Retailer",
    "host": "localhost", #if on other computer then IP Adress from other computer
    "port": 3306, #always 4 digits
    "user": "root",
    "password": password,
}
print("Confirm")

Connecting...


In [18]:
import pymysql
import pandas as pd
import plotly.express as px

In [19]:
def run_query_and_fetch(params, sql_query, cursorclass=pymysql.cursors.Cursor):
    """Connect to MySQL database and run a query"""
    conn = pymysql.connect(**params, cursorclass=cursorclass)
    with conn:
        with conn.cursor() as cursor:
            n_rows = cursor.execute(sql_query)
            print(f"Rows affected: {n_rows}")
            data = cursor.fetchall()
    return data

## Criterias for determining our customers

link to ref: https://towardsdatascience.com/know-your-customers-with-rfm-9f88f09433bc

* Champions: Bought recently, buy often and spend the most
* Loyal customers: Buy on a regular basis. Responsive to promotions.
* Potential loyalist: Recent customers with average frequency.
* Recent customers: Bought most recently, but not often.
* Promising: Recent shoppers, but haven’t spent much.
* Needs attention: Above average recency, frequency and monetary values. May not have bought very recently though.
* About to sleep: Below average recency and frequency. Will lose them if not reactivated.
* At risk: Some time since they’ve purchased. Need to bring them back!
* Can’t lose them: Used to purchase frequently but haven’t returned for a long time.
* Hibernating: Last purchase was long back and low number of orders. May be lost.

### Score Association

* Champions: High scores in all three RFM components (e.g., R_Score = 3, F_Score = 3, M_Score = 3).
* Loyal Customers: High Frequency and Monetary scores, regardless of Recency (e.g., F_Score = 3, M_Score = 3).
* Potential Loyalist: High Recency and moderate Frequency scores (e.g., R_Score = 3, F_Score = 2).
* Recent Customers: Very high Recency score, but lower on Frequency and Monetary (e.g., R_Score = 3, F_Score = 1 or 2, M_Score = 1 or 2).
* Promising: High Recency score, but low on Frequency and Monetary (e.g., R_Score = 3, F_Score = 1, M_Score = 1).
* Needs Attention: Moderate scores in all three components (e.g., R_Score = 2, F_Score = 2, M_Score = 2).
* About to Sleep: Low Recency and Frequency scores (e.g., R_Score = 1, F_Score = 1).
* At Risk: Low Recency score, but higher in Frequency and/or Monetary (e.g., R_Score = 1, F_Score = 2 or 3, M_Score = 2 or 3).
* Can't Lose Them: Low Recency score, but used to have high Frequency and/or Monetary scores (e.g., R_Score = 1, F_Score = 3, M_Score = 3).
* Hibernating: Low scores across all three components (e.g., R_Score = 1, F_Score = 1, M_Score = 1).

In [20]:
sql_query = """
WITH CustomerSales AS (
    SELECT
        ST.CustomerID,
        SUM(PA.Price * TI.ItemQuantity) AS TotalSalesValue
    FROM
        SalesTicket ST
    INNER JOIN TicketItems TI ON ST.SalesTicketID = TI.TicketID
    INNER JOIN PhoneAttributes PA ON TI.PhoneID = PA.PhoneID
    GROUP BY ST.CustomerID
),
CustomerRFM AS (
    SELECT
        ST.CustomerID,
        -- Recency: Days since last purchase
        DATEDIFF(CURRENT_DATE, MAX(ST.SaleDate)) AS Recency,
        -- Frequency: Number of purchases
        COUNT(DISTINCT ST.SalesTicketID) AS Frequency,
        CS.TotalSalesValue AS Monetary
    FROM
        SalesTicket ST
    INNER JOIN CustomerSales CS ON ST.CustomerID = CS.CustomerID
    GROUP BY ST.CustomerID, CS.TotalSalesValue
),
RFM_Scores AS (
    SELECT
        CR.CustomerID,
        CR.Recency,
        CR.Frequency,
        CR.Monetary,
        -- RFM scores
        NTILE(3) OVER (ORDER BY CR.Recency) AS R_Score,
        NTILE(3) OVER (ORDER BY CR.Frequency DESC) AS F_Score,
        NTILE(3) OVER (ORDER BY CR.Monetary DESC) AS M_Score
    FROM
        CustomerRFM CR
)
SELECT
    R.CustomerID,
    -- Real values
    R.Recency as nb_days_since_last_purchase,
    R.Frequency as nb_purchase,
    R.Monetary as sales_values,
    -- RFM scores
    R.R_Score,
    R.F_Score,
    R.M_Score,
    -- RFM Personas
    CASE
        WHEN R.R_Score = 3 AND R.F_Score = 3 AND R.M_Score = 3 THEN 'Champions'
        WHEN R.F_Score = 3 AND R.M_Score = 3 THEN 'Loyal Customers'
        WHEN R.R_Score = 3 AND R.F_Score = 2 THEN 'Potential Loyalist'
        WHEN R.R_Score = 3 AND R.F_Score IN (1, 2) AND R.M_Score IN (1, 2) THEN 'Recent Customers'
        WHEN R.R_Score = 3 AND R.F_Score = 1 AND R.M_Score = 1 THEN 'Promising'
        WHEN R.R_Score = 2 AND R.F_Score = 2 AND R.M_Score = 2 THEN 'Needs Attention'
        WHEN R.R_Score = 1 AND R.F_Score = 1 THEN 'About to Sleep'
        WHEN R.R_Score = 1 AND (R.F_Score = 2 OR R.F_Score = 3) AND (R.M_Score = 2 OR R.M_Score = 3) THEN 'At Risk'
        WHEN R.R_Score = 1 AND R.F_Score = 3 AND R.M_Score = 3 THEN 'Can’t Lose Them'
        ELSE 'Hibernating'
    END AS RFM_Personas
FROM
    RFM_Scores R

"""

In [21]:
data = run_query_and_fetch(
    MYSQL_PARAMS,
    sql_query,
    cursorclass=pymysql.cursors.DictCursor,
)
rfm_df = pd.DataFrame(data)

Rows affected: 100


In [22]:
rfm_df.head()

,CustomerID,nb_days_since_last_purchase,nb_purchase,sales_values,R_Score,F_Score,M_Score,RFM_Personas
0,39,205,2,8747.00,2,2,1,Hibernating
1,44,81,2,8397.00,1,1,1,About to Sleep
2,43,104,2,7477.00,1,1,1,About to Sleep
3,38,208,2,7347.00,2,2,1,Hibernating
4,42,150,2,6597.00,2,2,1,Hibernating


## Create Vizualisation in order to understand our customer base

In [38]:
persona_counts = rfm_df['RFM_Personas'].value_counts().reset_index()
persona_counts.columns = ['RFM_Personas', 'Count']

total_customers = persona_counts['Count'].sum()

# Calculate the percentage of each persona
persona_counts['Percentage'] = (persona_counts['Count'] / total_customers) * 100

# Create a treemap using the percentages
fig = px.treemap(persona_counts, path=['RFM_Personas'], values='Percentage',
                 title='Treemap of RFM Personas',
                 color='RFM_Personas',
                 )

fig.update_traces(textinfo="label+percent parent")  # This will show percentages on the treemap boxes
fig.show()



In [30]:
# Create a box plot
average_monetary = rfm_df.groupby('RFM_Personas')['sales_values'].mean().reset_index()

# Create a bar chart
fig = px.bar(average_monetary, x='RFM_Personas', y='sales_values',
             title='Average Monetary Values by RFM Personas',
             labels={'sales_values': 'Average Monetary Value'})
fig.show()

In [32]:
# Create a box plot
average_monetary = rfm_df.groupby('RFM_Personas')['nb_purchase'].mean().reset_index()

# Create a bar chart
fig = px.bar(average_monetary, x='RFM_Personas', y='nb_purchase',
             title='Average Number of Purchase by RFM Personas',
             labels={'sales_values': 'Average Monetary Value'})
fig.show()

In [33]:
# Create a box plot
average_monetary = rfm_df.groupby('RFM_Personas')['nb_days_since_last_purchase'].mean().reset_index()

# Create a bar chart
fig = px.bar(average_monetary, x='RFM_Personas', y='nb_days_since_last_purchase',
             title='Average Number of Days since last purchase by RFM Personas',
             labels={'sales_values': 'Average Monetary Value'})
fig.show()